In [1]:

import tensorflow as tf 

2024-09-05 20:57:14.691846: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-05 20:57:14.850413: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-05 20:57:15.141985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-05 20:57:15.494637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-05 20:57:15.540460: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-05 20:57:15.875158: I tensorflow/core/platform/cpu_feature_gu

<br>

### 1.1 `Usefull function to create input pipe line with tensorflow. `

- How to convert python list into TFdata.


<br>

In [4]:

# python list to TFdataset: 

daily_sale_number = [21,22,-108,31,-1,32,34,31]

tf_dataset = tf.data.Dataset.from_tensors(daily_sale_number)

tf_dataset

<_TensorDataset element_spec=TensorSpec(shape=(8,), dtype=tf.int32, name=None)>

In [10]:
# data specification :

tf_dataset.element_spec

TensorSpec(shape=(), dtype=tf.int32, name=None)

In [6]:

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sale_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<br>


# 1.2 : `How to print the tensorData.`

<br>

In [7]:
# print tensor with for loop

for sale in tf_dataset:
    print(sale)
    

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


2024-09-05 21:18:22.148603: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:

# convert tensor in numpy and print it 

for sale in tf_dataset:
    print(sale.numpy())
    

print()

for sale in tf_dataset.as_numpy_iterator():
    print(sale)
    

print()

# take some number: 
for sale in tf_dataset.take(3):
    print(sale)
    

21
22
-108
31
-1
32
34
31

21
22
-108
31
-1
32
34
31

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)


2024-09-05 21:23:08.056633: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<br>

# 1.3 `apply filter function: `

<br>

In [28]:

# filter into the TFdatast:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for i in tf_dataset:
    print(i.numpy())
    
    

21
22
31
32
34
31


<br>

# 1.4  `Multipy something with each of the elemnet in TFdataset. `

<br>

In [30]:

# access all of the element without for loop:

tf_dataset = tf_dataset.map(lambda x: x * (-1))
for i in tf_dataset:
    print(i.numpy()) 
    
    

-21
-22
-31
-32
-34
-31


<br>

# 1.5 `Batching dataset: `


<br>

In [34]:
# without batching: 
for i in tf_dataset:
    print(i.numpy())

-21
-22
-31
-32
-34
-31


In [31]:
# batch with size: 2 
for i in tf_dataset.batch(2):
    print(i)

tf.Tensor([-21 -22], shape=(2,), dtype=int32)
tf.Tensor([-31 -32], shape=(2,), dtype=int32)
tf.Tensor([-34 -31], shape=(2,), dtype=int32)


In [33]:
# batch with size: 3 
for i in tf_dataset.batch(3):
    print(i)

tf.Tensor([-21 -22 -31], shape=(3,), dtype=int32)
tf.Tensor([-32 -34 -31], shape=(3,), dtype=int32)


<br>

# 1.6 `Shuffle tensorData set: `

<br>




Imagine, you have a dataset: `[1, 2, 3, 4, 5, 6]`, then:

---

### How `ds.shuffle(buffer_size)` Works:

`dataset.shuffle(buffer_size=3)` will allocate a buffer of size 3 for picking random entries. This buffer will be connected to the source dataset. We could imagine it like this:

```
Random buffer
   |
   |   Source dataset where all other elements live
   |         |
   ↓         ↓
[1,2,3] <= [4,5,6]
```

Let's assume that entry `2` was taken from the random buffer. Free space is filled by the next element from the source buffer, that is `4`:

```
2 <= [1,3,4] <= [5,6]
```

We continue reading till nothing is left:

```
1 <= [3,4,5] <= [6]
5 <= [3,4,6] <= []
3 <= [4,6]   <= []
6 <= [4]     <= []
4 <= []      <= []
```

---

### How `ds.repeat()` Works:

As soon as all the entries are read from the dataset and you try to read the next element, the dataset will throw an error. That's where `ds.repeat()` comes into play. It will re-initialize the dataset, making it again like this:

```
[1,2,3] <= [4,5,6]
```

---

### What `ds.batch()` Produces:

The `ds.batch()` will take the first `batch_size` entries and make a batch out of them. So, a batch size of `3` for our example dataset will produce two batch records:

```
[2,1,5]
[3,6,4]
```

As we have a `ds.repeat()` before the batch, the generation of the data will continue. But the order of the elements will be different, due to the `ds.random()`. What should be taken into account is that `6` will never be present in the first batch, due to the size of the random buffer.


In [37]:

# now, let's try with an example: 

sample_data = [1,2,3,4,5,6]
ds = tf.data.Dataset.from_tensor_slices(sample_data)


# print tfData:
for i in ds:
    print(i.numpy())



1
2
3
4
5
6


In [39]:

# now use shuffle:
for i in ds.shuffle(buffer_size=3):
    print(i.numpy())
    
    

3
1
2
5
4
6




<br>

## `1.3, 1.4, 1.5, 1.6 apply all the transformation in a single line:` 

<br>


In [41]:


tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sale_number)

new_dataset = tf_dataset.filter(lambda x: x>0).map(lambda x: x*2).shuffle(3).batch(3)

for i in new_dataset:
    print(i)

Cause: could not parse the source code of <function <lambda> at 0x7ccb685f4d60>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x: x * 2

Match 1:
lambda x: x > 0

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ccb685f4d60>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x: x * 2

Match 1:
lambda x: x > 0

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ccb685f6d40>: found multiple definitions with identical signatures at the locat

In [42]:

# __________ use this to avoid warning __________ : 

@tf.autograph.experimental.do_not_convert
def process_data():
    tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sale_number)
    new_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda x: x * 2).shuffle(3).batch(3)
    
    for i in new_dataset:
        print(i)

process_data()


tf.Tensor([44 42 64], shape=(3,), dtype=int32)
tf.Tensor([68 62 62], shape=(3,), dtype=int32)
